# Credential Issuance and Presentation

<div class="alert alert-primary">
<b>🎯 OBJECTIVE</b><hr>
Demonstrate the process of issuing and presenting an ACDC (Authentic Chained Data Container) from an Issuer to a Holder, and subsequently from the Holder to a Verifier, using the Issuance and Presentation Exchange (IPEX) protocol with the Signify-ts library.
</div>

<div class="alert alert-info">
<b>ℹ️ NOTE</b><hr>
This section utilizes utility functions (from <code>./scripts_ts/utils.ts</code>) to quickly establish the necessary preconditions for credential issuance and presentation. The detailed steps for client initialization, AID creation, end role assignment, and OOBI resolution were covered in the "Keria-Signify Connecting Controllers" notebook. Here, we provide a high-level recap of what these utility functions accomplish.
</div>

## Client setup

The setup process, streamlined by the utility functions, performs the following key actions:

* **Signify Library Initialization**: Ensures the underlying cryptographic components of Signify-ts are ready.
* **Client Initialization & Connection**: Three `SignifyClient` instances are created—one each for an Issuer, a Holder, and a Verifier. Each client is bootstrapped and connected to its Keria agent.
* **AID Creation**: Each client (Issuer, Holder, Verifier) creates a primary AID using default arguments.
* **End Role Assignment**: An `agent` end role is assigned to each client's Keria Agent AID. 
* **OOBI Generation and Resolution (Client-to-Client)**:
    * OOBIs are generated for the Issuer, Holder, and Verifier AIDs, specifically for the `'agent'` role.
    * Secure communication channels are established by resolving these OOBIs:
        * Issuer's client resolves the Holder's OOBI.
        * Holder's client resolves the Issuer's OOBI.
        * Verifier's client resolves the Holder's OOBI.
        * Holder's client resolves the Verifier's OOBI.
* **Schema OOBI Resolution**: The Issuer, Holder, and Verifier clients all resolve the OOBI for the "EventPass" schema (SAID: `EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK`). This schema is hosted on the schema server (vLEI-Server in this context). Resolving the schema OOBI ensures all parties have the correct and verifiable schema definition necessary to understand and validate the credential.

The code block below executes this setup.

In [1]:
import { randomPasscode, Serder} from 'npm:signify-ts';
import { initializeSignify, 
         initializeAndConnectClient,
         createNewAID,
         addEndRoleForAID,
         generateOOBI,
         resolveOOBI,
         createTimestamp,
         DEFAULT_IDENTIFIER_ARGS,
         DEFAULT_TIMEOUT_MS,
         DEFAULT_DELAY_MS,
         DEFAULT_RETRIES,
         ROLE_AGENT,
         IPEX_GRANT_ROUTE,
         IPEX_ADMIT_ROUTE,
         IPEX_APPLY_ROUTE,
         IPEX_OFFER_ROUTE,
         SCHEMA_SERVER_HOST
       } from './scripts_ts/utils.ts';

// Clients setup
// Initialize Issuer, Holder and Verifier CLients, Create AIDs for each one, assign 'agent' role to the AIDs
// generate and resolve OOBIs 

// Issuer Client
const issuerBran = randomPasscode()
const issuerAidAlias = 'issuerAid'
const { client: issuerClient } = await initializeAndConnectClient(issuerBran)
const { aid: issuerAid} = await createNewAID(issuerClient, issuerAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(issuerClient, issuerAidAlias, ROLE_AGENT);
const issuerOOBI = await generateOOBI(issuerClient, issuerAidAlias, ROLE_AGENT);

// Holder Client
const holderBran = randomPasscode()
const holderAidAlias = 'holderAid'
const { client: holderClient } = await initializeAndConnectClient(holderBran)
const { aid: holderAid} = await createNewAID(holderClient, holderAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(holderClient, holderAidAlias, ROLE_AGENT);
const holderOOBI = await generateOOBI(holderClient, holderAidAlias, ROLE_AGENT);

// Verifier Client
const verifierBran = randomPasscode()
const verifierAidAlias = 'verifierAid'
const { client: verifierClient } = await initializeAndConnectClient(verifierBran)
const { aid: verifierAid} = await createNewAID(verifierClient, verifierAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(verifierClient, verifierAidAlias, ROLE_AGENT);
const verifierOOBI = await generateOOBI(verifierClient, verifierAidAlias, ROLE_AGENT);

// Clients OOBI Resolution
// Resolve OOBIs to establish connections Issuer-Holder, Holder-Verifier
const issuerContactAlias = 'issuerContact';
const holderContactAlias = 'holderContact';
const verifierContactAlias = 'verifierContact';

await resolveOOBI(issuerClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, issuerOOBI, issuerContactAlias);
await resolveOOBI(verifierClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, verifierOOBI, verifierContactAlias);

// Schemas OOBI Resolution
// Resolve the Schemas from the Schema Server (VLEI-Server)
const schemaContactAlias = 'schemaContact';
const schemaSaid = 'EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK';
const schemaOOBI = `http://vlei-server:7723/oobi/${schemaSaid}`;

await resolveOOBI(issuerClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(holderClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(verifierClient, schemaOOBI, schemaContactAlias);

console.log("Client setup and OOBI resolutions complete.");

Using Passcode (bran): APtHi0hwEwUXnB71iLQqD
Client boot process initiated with Keria agent.
  Client AID Prefix:  EAvKiXiZPTLbrMVuxn43I4nI0lb7vpkam2PV2miy-6SH
  Agent AID Prefix:   EJTnz7wre12XSaW0bTmdSE9UGNUNusIPKeMrp0OAxqlJ
Initiating AID inception for alias: issuerAid
Successfully created AID with prefix: EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA
Assigning 'agent' role to Keria Agent EJTnz7wre12XSaW0bTmdSE9UGNUNusIPKeMrp0OAxqlJ for AID alias issuerAid
Successfully assigned 'agent' role for AID alias issuerAid.
Generating OOBI for AID alias issuerAid with role agent
Generated OOBI URL: http://keria:3902/oobi/EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA/agent/EJTnz7wre12XSaW0bTmdSE9UGNUNusIPKeMrp0OAxqlJ
Using Passcode (bran): AuOXDIgvNyPN7wKoOaS0A
Client boot process initiated with Keria agent.
  Client AID Prefix:  EEROXTvt0tSq2s3KZdsR5T7Rut8XQTa08tmHdoctkFVj
  Agent AID Prefix:   EE1i8cTrS3aO4pxGy3tREaE8jUeQBk5qAF2Ta29VFQVR
Initiating AID inception for alias: holderAid
Successfu

## Credential Issuance

With the clients set up and connected, you can proceed with the credential issuance workflow. This involves the Issuer creating a credential and transferring it to the Holder using the IPEX protocol.

### Create the Credential Registry 

Before an Issuer can issue credentials, it needs a Credential Registry. In KERI, a Credential Registry is implemented using a Transaction Event Log (TEL). This TEL is a secure, hash-linked log, managed by the Issuer's AID, specifically for tracking the lifecycle of credentials it issues—such as their issuance and revocation status. The registry itself is identified by a SAID (Self-Addressing Identifier) derived from its inception event (vcp event type for registry inception). The TEL's history is anchored to the Issuer's main Key Event Log (KEL), ensuring that all changes to the registry's state are cryptographically secured by the Issuer's controlling keys. This anchoring is achieved by including a digest of the TEL event in a KEL event.

Use the code below to let the Issuer client create this registry. A human-readable name (issuerRegistryName) is used to reference it within the client.

In [2]:
//Create Issuer credential Registry
const issuerRegistryName = 'issuerRegistry' // Human readable identifier for the Registry

// Initiate registry creation
const createRegistryResult = await issuerClient
    .registries()
    .create({ name: issuerAidAlias, registryName: issuerRegistryName });

// Get the operation details
const createRegistryOperation = await createRegistryResult.op();

// Wait for the operation to complete
const createRegistryResponse = await issuerClient
    .operations()
    .wait(createRegistryOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation from the agent's list
await issuerClient.operations().delete(createRegistryOperation.name);

console.log(`Registry '${issuerRegistryName}' created for Issuer AID ${issuerAid.i}.`);
console.log("Registry creation response:", JSON.stringify(createRegistryResponse.response, null, 2));

// Listing Registries to confirm creation and retrieve its SAID (regk)
const issuerRegistries = await issuerClient.registries().list(issuerAidAlias);
const issuerRegistry = issuerRegistries[0]
console.log(`Registry: Name='${issuerRegistry.name}', SAID (regk)='${issuerRegistry.regk}'`);

Registry 'issuerRegistry' created for Issuer AID EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA.
Registry creation response: {
  "anchor": {
    "i": "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
    "s": "0",
    "d": "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U"
  }
}
Registry: Name='issuerRegistry', SAID (regk)='EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U'


### Retrieve schemas definition

The Issuer needs the definition of the schema against which they intend to issue a credential. Since the schema OOBI was resolved during the setup phase, the schema definition can now be retrieved from the Keria agent's cache using its SAID. You will reuse the `EventPass` schema (SAID: `EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK`) from previous KLI examples.

In [3]:
// Retrieve Schemas
const issuerSchema = await issuerClient.schemas().get(schemaSaid);
console.log(issuerSchema)

{
  "$id": "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
  "$schema": "http://json-schema.org/draft-07/schema#",
  title: "EventPass",
  description: "Event Pass Schema",
  type: "object",
  credentialType: "EventPassCred",
  version: "1.0.0",
  properties: {
    v: { description: "Credential Version String", type: "string" },
    d: { description: "Credential SAID", type: "string" },
    u: { description: "One time use nonce", type: "string" },
    i: { description: "Issuer AID", type: "string" },
    ri: { description: "Registry SAID", type: "string" },
    s: { description: "Schema SAID", type: "string" },
    a: {
      oneOf: [
        { description: "Attributes block SAID", type: "string" },
        {
          "$id": "ELppbffpWEM-uufl6qpVTcN6LoZS2A69UN4Ddrtr_JqE",
          description: "Attributes block",
          type: "object",
          properties: [Object],
          additionalProperties: false,
          required: [Array]
        }
      ]
    }
  },
  additionalPropert

### Issue Credential

Now the Issuer creates the actual ACDC. This involves:

1. Defining the credentialClaims – the specific attribute values for this instance of the `EventPass` credential.
2. Calling `issuerClient.credentials().issue()`. This method takes the Issuer's AID alias and an object specifying:
    - `ri`: The SAID of the Credential Registry (`issuerRegistry.regk`) where this credential's issuance will be recorded.
    - `s`: The SAID of the schema (`schemaSaid`) this credential adheres to.
    - `a`: An attributes block containing:
      - `i`: The AID of the Issuee (the Holder, holderAid.i).
      - The actual `credentialClaims`.

This `issue` command creates the ACDC locally within the Issuer's client and records an issuance event (e.g., `iss`) in the specified registry's TEL. The SAID of the newly created credential is then extracted from the response.

Use the code below to perform these actions.

In [4]:
// Issue Credential

const credentialClaims = {
    "eventName":"GLEIF Summit",
    "accessLevel":"staff",
    "validDate":"2026-10-01"
}

const issueResult = await issuerClient
    .credentials()
    .issue(
        issuerAidAlias,
        {
            ri: issuerRegistry.regk, //Registry Identifier (not the alias)
            s: schemaSaid,           // Schema identifier
            a: {                     // Attributes block
                i: holderAid.i,      // Isuue or credential subject 
                ...credentialClaims  // The actual claims data                 
            }
        });

// Issuance is an asynchronous operation.
const issueOperation = await issueResult.op;

// Wait for the issuance operation to complete.
const issueResponse = await issuerClient
    .operations()
    .wait(issueOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await issuerClient.operations().delete(issueOperation.name);

// Extract the SAID of the newly created credential from the response.
// This SAID uniquely identifies this specific ACDC instance.
const credentialSaid = issueResponse.response.ced.d

// Display the issued credential from the Issuer's perspective.
const issuerCredential = await issuerClient.credentials().get(credentialSaid);
console.log(issuerCredential)

{
  sad: {
    v: "ACDC10JSON0001c4_",
    d: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
    i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
    ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
    s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
    a: {
      d: "ELx-jzWvuG7U285kTpNF3nWhKRQSh6Bk_RQLAGqIH1Oq",
      i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
      eventName: "GLEIF Summit",
      accessLevel: "staff",
      validDate: "2026-10-01",
      dt: "2025-05-20T05:21:04.727000+00:00"
    }
  },
  atc: "-IABEGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g0AAAAAAAAAAAAAAAAAAAAAAAEGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
  iss: {
    v: "KERI10JSON0000ed_",
    t: "iss",
    d: "ECmdsIyE9mIMjOz-1rt6zqo7LS4xrfR_U84N-aWDHDiW",
    i: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
    s: "0",
    ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
    dt: "2025-05-20T05:21:04.727000+00:00"
  },
  issatc: "-VAS-GAB0AAAAAAAAAAAAAAAAAAAAAACEMjRD-sCEu6wSApyx99FsY

### Issuer Ipex Grant

The credential has been created but currently resides only with the Issuer. To transfer it to the Holder, the Issuer initiates an IPEX (Issuance and Presentation Exchange) grant. This process uses KERI `exn` (exchange) messages. The grant message effectively offers the credential to the Holder. 

The `issuerClient.ipex().grant()` method prepares the grant message, including the ACDC itself (`acdc`), the issuance event from the registry (`iss`), and the anchoring event from the Issuer's KEL (`anc`) along with its signatures (`ancAttachment`).
Then, `issuerClient.ipex().submitGrant()` sends this packaged grant message to the Holder's Keria agent.

Use the code below to perform the ipex grant.

In [5]:
// Ipex Grant

const [grant, gsigs, gend] = await issuerClient.ipex().grant({
    senderName: issuerAidAlias,
    acdc: new Serder(issuerCredential.sad), // The ACDC (Verifiable Credential) itself
    iss: new Serder(issuerCredential.iss),  // The issuance event from the credential registry (TEL event)
    anc: new Serder(issuerCredential.anc),  // The KEL event anchoring the TEL issuance event
    ancAttachment: issuerCredential.ancatc, // Signatures for the KEL anchoring event
    recipient: holderAid.i,                 // AID of the Holder
    datetime: createTimestamp(),            // Timestamp for the grant message
});

// Issuer submits the prepared grant message to the Holder.
// This sends an 'exn' message to the Holder's Keria agent.
const submitGrantOperation = await issuerClient
    .ipex()
    .submitGrant(
        issuerAidAlias,  // Issuer's AID alias
        grant,           // The grant message payload
        gsigs,           // Signatures for the grant message
        gend,            // Endorsements for the grant message
        [holderAid.i]    // List of recipient AIDs
    );

// Wait for the submission operation to complete.
const submitGrantResponse = await issuerClient
    .operations()
    .wait(submitGrantOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await issuerClient.operations().delete(submitGrantOperation.name);

### Holder Credential State

The Holder can proactively check the status of a credential in the Issuer's registry if they know the registry's SAID (`issuerRegistry.regk`) and the credential's SAID (`issuerCredential.sad.d`). This query demonstrates how a party can verify the status directly from the TEL.

In [6]:
// The flow transitions from the Issuer to the Holder.
// A delay and retry mechanism is added to allow time for Keria agents and witnesses
// to propagate the credential issuance information.

let credentialState;

// Retry loop to fetch credential state from the Holder's perspective.
for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        // Holder's client queries the state of the credential in the Issuer's registry.
        credentialState = await holderClient.credentials().state(issuerRegistry.regk, issuerCredential.sad.d)
        break;
    }
    catch (error){    
         console.log(`[Retry] failed to get credential state on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached for getting credential state.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

console.log(credentialState) // Displays the status (e.g., issued, revoked)

[Retry] failed to get credential state on attempt #1 of 5
[Retry] Waiting 5000ms before next attempt...
{
  vn: [ 1, 0 ],
  i: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
  s: "0",
  d: "ECmdsIyE9mIMjOz-1rt6zqo7LS4xrfR_U84N-aWDHDiW",
  ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
  ra: {},
  a: { s: 2, d: "EMjRD-sCEu6wSApyx99FsYL2mZGOwMGrmlv6l5X2bBSH" },
  dt: "2025-05-20T05:21:04.727000+00:00",
  et: "iss"
}


### Holder Grant notification and Exchange retrieve

The Holder's Keria agent will receive the grant `exn` message sent by the Issuer. The Holder's client can list its notifications to find this incoming grant. The notification will contain the SAID of the `exn` message (`grantNotification.a.d`), which can then be used to retrieve the full details of the grant exchange from the Holder's client.



In [7]:
// Holder waits for Grant notification

let notifications;

// Retry loop to fetch notifications.
for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        // List notifications, filtering for unread IPEX_GRANT_ROUTE messages.
        notifications = await holderClient.notifications().list(
            (n) => n.a.r === IPEX_GRANT_ROUTE && n.r === false // n.r is 'read' status
        );
        if(notifications.notes.length === 0){ 
            throw new Error("Grant notification not found"); // Throw error to trigger retry
        }
        break;     
    }
    catch (error){    
         console.log(`[Retry] Grant notification not found on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached for grant notification.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const grantNotification = notifications.notes[0]  // Assuming only one grant notification for simplicity

console.log(grantNotification) // Displays the notification details

// Retrieve the full IPEX grant exchange details using the SAID from the notification.
// The 'exn' field in the exchange will contain the actual credential data.
const grantExchange = await holderClient.exchanges().get(grantNotification.a.d);

console.log(grantExchange) // Displays the content of the grant message

{
  i: "0AB7Y4hnihAuWLpnwuFWNa4c",
  dt: "2025-05-20T05:21:06.085816+00:00",
  r: false,
  a: {
    r: "/exn/ipex/grant",
    d: "EHGvUCLJbgQ0fcJjCnQ8U5_upC25elKB_eZX-XGJ16T2",
    m: ""
  }
}
{
  exn: {
    v: "KERI10JSON00057f_",
    t: "exn",
    d: "EHGvUCLJbgQ0fcJjCnQ8U5_upC25elKB_eZX-XGJ16T2",
    i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
    rp: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
    p: "",
    dt: "2025-05-20T05:21:05.715000+00:00",
    r: "/ipex/grant",
    q: {},
    a: { i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy", m: "" },
    e: {
      acdc: {
        v: "ACDC10JSON0001c4_",
        d: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
        i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
        ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
        s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
        a: {
          d: "ELx-jzWvuG7U285kTpNF3nWhKRQSh6Bk_RQLAGqIH1Oq",
          i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
    

### Holder Admits Grant and marks notification

Upon receiving and reviewing the grant, the Holder decides to accept (`admit`) the credential. This involves:
- Preparing an `admit` `exn` message using `holderClient.ipex().admit()`.
- Submitting this `admit` message back to the Issuer using `holderClient.ipex().submitAdmit()`.
- Marking the original grant notification as read.
- The Holder's client then processes the admitted credential, verifying its signatures, schema, and status against the Issuer's KEL and TEL, and stores it locally.

In [8]:
// Holder admits (accepts) the IPEX grant.

// Prepare the IPEX admit message.
const [admit, sigs, aend] = await holderClient.ipex().admit({
    senderName: holderAidAlias,       // Alias of the Holder's AID
    message: '',                      // Optional message to include in the admit
    grantSaid: grantNotification.a.d!,// SAID of the grant 'exn' message being admitted
    recipient: issuerAid.i,           // AID of the Issuer
    datetime: createTimestamp(),      // Timestamp for the admit message
});

// Holder submits the prepared admit message to the Issuer.
const admitOperation = await holderClient
    .ipex()
    .submitAdmit(holderAidAlias, admit, sigs, aend, [issuerAid.i]);

// Wait for the submission operation to complete.
const admitResponse = await holderClient
    .operations()
    .wait(admitOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await holderClient.operations().delete(admitOperation.name);

// Holder marks the grant notification as read.
await holderClient.notifications().mark(grantNotification.i);
console.log("Holder's notifications after marking grant as read:");
console.log(await holderClient.notifications().list());

// Holder can now get the credential from their local store.
// This implies the client has processed, verified, and stored it upon admission.
const holderReceivedCredential = await holderClient.credentials().get(issuerCredential.sad.d);
console.log("Credential as stored by Holder:");
console.log(holderReceivedCredential);


Holder's notifications after marking grant as read:
{
  start: 0,
  end: 0,
  total: 1,
  notes: [
    {
      i: "0AB7Y4hnihAuWLpnwuFWNa4c",
      dt: "2025-05-20T05:21:06.085816+00:00",
      r: true,
      a: {
        r: "/exn/ipex/grant",
        d: "EHGvUCLJbgQ0fcJjCnQ8U5_upC25elKB_eZX-XGJ16T2",
        m: ""
      }
    }
  ]
}
Credential as stored by Holder:
{
  sad: {
    v: "ACDC10JSON0001c4_",
    d: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
    i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
    ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
    s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
    a: {
      d: "ELx-jzWvuG7U285kTpNF3nWhKRQSh6Bk_RQLAGqIH1Oq",
      i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
      eventName: "GLEIF Summit",
      accessLevel: "staff",
      validDate: "2026-10-01",
      dt: "2025-05-20T05:21:04.727000+00:00"
    }
  },
  atc: "-IABEGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g0AAAAAAAAAAAAAAAAAAAAAAAEGrqQEY2hp5xKlWRRwFp

### Issuer Admit Notification

The Issuer, in turn, will receive a notification that the Holder has admitted the credential. The Issuer's client lists its notifications, finds the `admit` message, and marks it as read. This completes the issuance loop.

In [9]:
// Issuer retrieves the Admit notification from the Holder.

let issuerAdmitNotifications;

// Retry loop for the Issuer to receive the admit notification.
for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        // List notifications, filtering for unread IPEX_ADMIT_ROUTE messages.
        issuerAdmitNotifications = await issuerClient.notifications().list(
            (n) => n.a.r === IPEX_ADMIT_ROUTE && n.r === false
        );
        if(issuerAdmitNotifications.notes.length === 0){ 
            throw new Error("Admit notification not found"); // Throw error to trigger retry
        }
        break; // Exit loop if notification found
    }
    catch (error){    
         console.log(`[Retry] Admit notification not found for Issuer on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached for Issuer's admit notification.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const admitNotificationForIssuer = issuerAdmitNotifications.notes[0] // Assuming one notification

// Issuer marks the admit notification as read.
await issuerClient.notifications().mark(admitNotificationForIssuer.i);
console.log("Issuer's notifications after marking admit as read:");
console.log(await issuerClient.notifications().list());

Issuer's notifications after marking admit as read:
{
  start: 0,
  end: 0,
  total: 1,
  notes: [
    {
      i: "0ABrCJzDJ4GywX5L_JgGl1Nq",
      dt: "2025-05-20T05:21:11.662675+00:00",
      r: true,
      a: {
        r: "/exn/ipex/admit",
        d: "EOGCTIsfh5lyp3Y15VGIR35dt1VYA99fM47ek0R_jgsk",
        m: ""
      }
    }
  ]
}


### Cleanup

Once the IPEX flow for issuance is complete and notifications have been processed, both parties can optionally delete these notifications from their local client stores.

In [10]:
// Issuer Remove Admit Notification from their list
await issuerClient.notifications().delete(admitNotificationForIssuer.i);
console.log("Issuer's notifications after deleting admit notification:");
console.log(await issuerClient.notifications().list());

// Holder Remove Grant Notification from their list
await holderClient.notifications().delete(grantNotification.i);
console.log("Holder's notifications after deleting grant notification:");
console.log(await holderClient.notifications().list());

Issuer's notifications after deleting admit notification:
{ start: 0, end: 0, total: 0, notes: [] }
Holder's notifications after deleting grant notification:
{ start: 0, end: 0, total: 0, notes: [] }


## Presentation

Now that the Holder possesses the credential, they can present it to a Verifier. This workflow also uses IPEX, but typically starts with the Verifier requesting a presentation.

Below are presented the code snipets you need to follow to do the presentation.

### Verifier Apply

The Verifier initiates the presentation process by sending an IPEX apply message. This `apply` message is an `exn` message specifying the criteria for the credential they are requesting. This includes the `schemaSaid` and can include specific attributes the credential must have.

In [11]:
// Verifier Ipex Apply (Presentation request)

// Prepare the IPEX apply message.
const [apply, sigsApply, _endApply] = await verifierClient.ipex().apply({
    senderName: verifierAidAlias,     // Alias of the Verifier's AID
    schemaSaid: schemaSaid,           // SAID of the schema for the requested credential
    attributes: { eventName:'GLEIF Summit' }, // Specific attributes the credential should have
    recipient: holderAid.i,           // AID of the Holder being asked for the presentation
    datetime: createTimestamp(),      // Timestamp for the apply message
});

// Verifier submits the prepared apply message to the Holder.
const applyOperation = await verifierClient
    .ipex()
    .submitApply(verifierAidAlias, apply, sigsApply, [holderAid.i]);

// Wait for the submission operation to complete.
const applyResponse = await verifierClient
    .operations()
    .wait(applyOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await verifierClient.operations().delete(applyOperation.name);

### Holder Apply Notification and Exchange

Holder Apply Notification and Exchange
The Holder receives a notification for the Verifier's `apply` request. They retrieve the details of this request from the exchange message. After processing, the Holder marks the notification as read.

In [12]:
// Holder receives the IPEX apply notification from the Verifier.

let holderApplyNotifications;

// Retry loop for the Holder to receive the apply notification.
for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        // List notifications, filtering for unread IPEX_APPLY_ROUTE messages.
        holderApplyNotifications = await holderClient.notifications().list(
            (n) => n.a.r === IPEX_APPLY_ROUTE && n.r === false
        );
        if(holderApplyNotifications.notes.length === 0){ 
            throw new Error("Apply notification not found"); // Throw error to trigger retry
        }
        break; // Exit loop if notification found
    }
    catch (error){    
         console.log(`[Retry] Apply notification not found for Holder on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached for Holder's apply notification.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const applyNotificationForHolder = holderApplyNotifications.notes[0] // Assuming one notification

console.log("Holder received Apply Notification:");
console.log(applyNotificationForHolder);

// Retrieve the full IPEX apply exchange details.
const applyExchange = await holderClient.exchanges().get(applyNotificationForHolder.a.d);
console.log("Details of Apply Exchange received by Holder:");
console.log(applyExchange);

// Extract the SAID of the apply 'exn' message for use in the offer.
const applyExchangeSaid = applyExchange.exn.d;

// Holder marks the apply notification as read.
await holderClient.notifications().mark(applyNotificationForHolder.i);
console.log("Holder's notifications after marking apply as read:");
console.log(await holderClient.notifications().list());



[Retry] Apply notification not found for Holder on attempt #1 of 5
[Retry] Waiting 5000ms before next attempt...
Holder received Apply Notification:
{
  i: "0ACt9GVPENd6kB_TTg9TSN2l",
  dt: "2025-05-20T05:21:12.178333+00:00",
  r: false,
  a: {
    r: "/exn/ipex/apply",
    d: "EJkr7iYLjhpam57TBHNKKuHPZaFEhuX8v0ln8qTlZmS7",
    m: ""
  }
}
Details of Apply Exchange received by Holder:
{
  exn: {
    v: "KERI10JSON0001a0_",
    t: "exn",
    d: "EJkr7iYLjhpam57TBHNKKuHPZaFEhuX8v0ln8qTlZmS7",
    i: "EIx2GoLOwDmml9QHzbeD_zC5qy1-yOWgZ2guM-0iaUh7",
    rp: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
    p: "",
    dt: "2025-05-20T05:21:11.809000+00:00",
    r: "/ipex/apply",
    q: {},
    a: {
      i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
      m: "",
      s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
      a: { eventName: "GLEIF Summit" }
    },
    e: {}
  },
  pathed: {}
}
Holder's notifications after marking apply as read:
{
  start: 0,
  end: 0,
  total: 1,
  not

### Holder Find Matching Credential

The Holder now needs to find a credential in their possession that satisfies the Verifier's `apply` request (matches the schema SAID and any specified attributes). The code below constructs a filter based on the `applyExchange` data and uses it to search the Holder's credentials.

In [13]:
// The apply operation from the Verifier asks for a specific credential 
// (matching schema and attribute values).
// This code snippet creates a credential filter based on the criteria
// from the applyExchange message received by the Holder.

let filter: { [x: string]: any } = { '-s': applyExchange.exn.a.s }; // Filter by schema SAID
// Add attribute filters from the apply request
for (const key in applyExchange.exn.a.a) { // 'a.a' contains the requested attributes
    filter[`-a-${key}`] = applyExchange.exn.a.a[key];
}

console.log("Constructed filter for matching credentials:");
console.log(filter);

// Holder lists credentials matching the filter.
const matchingCredentials = await holderClient.credentials().list({ filter });

console.log("Matching credentials found by Holder:");
console.log(matchingCredentials); // Should list the EventPass credential issued earlier

Constructed filter for matching credentials:
{
  "-s": "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
  "-a-eventName": "GLEIF Summit"
}
Matching credentials found by Holder:
[
  {
    sad: {
      v: "ACDC10JSON0001c4_",
      d: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
      i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
      ri: "EOnm7O6sfm0_YBVJyyFhTbXhdReMxRjjt9IQNbojd24U",
      s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
      a: {
        d: "ELx-jzWvuG7U285kTpNF3nWhKRQSh6Bk_RQLAGqIH1Oq",
        i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
        eventName: "GLEIF Summit",
        accessLevel: "staff",
        validDate: "2026-10-01",
        dt: "2025-05-20T05:21:04.727000+00:00"
      }
    },
    atc: "-IABEGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g0AAAAAAAAAAAAAAAAAAAAAAAEGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
    iss: {
      v: "KERI10JSON0000ed_",
      t: "iss",
      d: "ECmdsIyE9mIMjOz-1rt6zqo7LS4xrfR_U84N-aWDHDiW",
      i: "EGrqQE

### Holder offer

Assuming a matching credential is found, the Holder prepares an IPEX offer message. This `offer` includes the ACDC they are presenting. This is sent back to the Verifier.

In [14]:
// Holder prepares and submits an IPEX offer message with the matching credential.

// Prepare the IPEX offer message.
const [offer, sigsOffer, endOffer] = await holderClient.ipex().offer({
    senderName: holderAidAlias,                   // Alias of the Holder's AID
    recipient: verifierAid.i,                     // AID of the Verifier
    acdc: new Serder(matchingCredentials[0].sad), // The ACDC being offered (first matching credential)
    applySaid: applyExchangeSaid,                 // SAID of the Verifier's apply 'exn' message this offer is responding to
    datetime: createTimestamp(),                  // Timestamp for the offer message
});

// Holder submits the prepared offer message to the Verifier.
const offerOperation = await holderClient
    .ipex()
    .submitOffer(holderAidAlias, offer, sigsOffer, endOffer, [
        verifierAid.i, // Recipient AID
    ]);

// Wait for the submission operation to complete.
const offerResponse = await holderClient
    .operations()
    .wait(offerOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await holderClient.operations().delete(offerOperation.name);
console.log("Holder submitted IPEX Offer to Verifier.");



Holder submitted IPEX Offer to Verifier.


### Verifier - Handle Offer Notification and Agree 

The Verifier receives a notification for the Holder's `offer`. The Verifier retrieves the exchange details and marks the notification.

In [15]:
// Verifier receives the IPEX offer notification from the Holder.

let verifierOfferNotifications;

// Retry loop for the Verifier to receive the offer notification.
for (let attempt = 1; attempt <= DEFAULT_RETRIES ; attempt++) {
    try{
        // List notifications, filtering for unread IPEX_OFFER_ROUTE messages.
        verifierOfferNotifications = await verifierClient.notifications().list(
            (n) => n.a.r === IPEX_OFFER_ROUTE && n.r === false
        );
        if(verifierOfferNotifications.notes.length === 0){ 
            throw new Error("Offer notification not found"); // Throw error to trigger retry
        }
        break; // Exit loop if notification found
    }
    catch (error){    
         console.log(`[Retry] Offer notification not found for Verifier on attempt #${attempt} of ${DEFAULT_RETRIES}`);
         if (attempt === DEFAULT_RETRIES) {
             console.error(`[Retry] Max retries (${DEFAULT_RETRIES}) reached for Verifier's offer notification.`);
             throw error; 
         }
         console.log(`[Retry] Waiting ${DEFAULT_DELAY_MS}ms before next attempt...`);
         await new Promise(resolve => setTimeout(resolve, DEFAULT_DELAY_MS));
    }
}

const offerNotificationForVerifier = verifierOfferNotifications.notes[0]; // Assuming one notification

console.log("Verifier received Offer Notification:");
console.log(offerNotificationForVerifier);

// Retrieve the full IPEX offer exchange details.
const offerExchange = await verifierClient.exchanges().get(offerNotificationForVerifier.a.d);
console.log("Details of Offer Exchange received by Verifier:");
console.log(offerExchange); // This will contain the ACDC presented by the Holder

// Extract the SAID of the offer 'exn' message for use in the agree.
let offerExchangeSaid = offerExchange.exn.d;

// Verifier marks the offer notification as read.
await verifierClient.notifications().mark(offerNotificationForVerifier.i);
console.log("Verifier's notifications after marking offer as read:");
console.log(await verifierClient.notifications().list());

[Retry] Offer notification not found for Verifier on attempt #1 of 5
[Retry] Waiting 5000ms before next attempt...
Verifier received Offer Notification:
{
  i: "0ADlYEvZme3auJFL8jwU8tpp",
  dt: "2025-05-20T05:21:17.679401+00:00",
  r: false,
  a: {
    r: "/exn/ipex/offer",
    d: "EGq91o0xXR8EW99bEEWzJ6FXgZAx51MtSRvOfy_KSemk",
    m: ""
  }
}
Details of Offer Exchange received by Verifier:
{
  exn: {
    v: "KERI10JSON000376_",
    t: "exn",
    d: "EGq91o0xXR8EW99bEEWzJ6FXgZAx51MtSRvOfy_KSemk",
    i: "EDMrSobA6cQtHN89mY0FTFlH89APILUkt9qr8awpAnwy",
    rp: "EIx2GoLOwDmml9QHzbeD_zC5qy1-yOWgZ2guM-0iaUh7",
    p: "EJkr7iYLjhpam57TBHNKKuHPZaFEhuX8v0ln8qTlZmS7",
    dt: "2025-05-20T05:21:17.309000+00:00",
    r: "/ipex/offer",
    q: {},
    a: { i: "EIx2GoLOwDmml9QHzbeD_zC5qy1-yOWgZ2guM-0iaUh7", m: "" },
    e: {
      acdc: {
        v: "ACDC10JSON0001c4_",
        d: "EGrqQEY2hp5xKlWRRwFp_pnZ_M8H77Bs08cIXFXAix7g",
        i: "EHeZgfT1dP23OiDWXrj7zr7PDDCUhMKZq3PnlCvOYaAA",
        ri: "

### Verifier - agree

Finally, the Verifier, after validating the offered credential (which signify-ts does implicitly upon processing the offer and preparing the agree), sends an IPEX agree message back to the Holder. This confirms successful receipt and validation of the presentation.

In [16]:
// Verifier prepares and submits an IPEX agree message.

// Prepare the IPEX agree message.
const [agree, sigsAgree, _endAgree] = await verifierClient.ipex().agree({
    senderName: verifierAidAlias, // Alias of the Verifier's AID
    recipient: holderAid.i,       // AID of the Holder
    offerSaid: offerExchangeSaid, // SAID of the Holder's offer 'exn' message this agree is responding to
    datetime: createTimestamp(),  // Timestamp for the agree message
});

// Verifier submits the prepared agree message to the Holder.
const agreeOperation = await verifierClient
    .ipex()
    .submitAgree(verifierAidAlias, agree, sigsAgree, [holderAid.i]);

// Wait for the submission operation to complete.
const agreeResponse = await verifierClient
    .operations()
    .wait(agreeOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

// Clean up the operation.
await verifierClient.operations().delete(agreeOperation.name);
console.log("Verifier submitted IPEX Agree to Holder, completing the presentation exchange.");

// At this point, the Verifier has successfully received and validated the credential.
// The Verifier's client would have stored the presented credential details if needed.

Verifier submitted IPEX Agree to Holder, completing the presentation exchange.
